<a href="https://colab.research.google.com/github/iguchi-lab/jjj_CRAC/blob/master/CENTRAL_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive のマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# フォルダの設定

In [ ]:
import os
import sys
import numpy as np
import pandas as pd

In [ ]:
sys.path.append('/content/drive/My Drive/Colab Notebooks/全館暖冷房WG/')
%cd /content/drive/My Drive/Colab Notebooks/全館暖冷房WG/
%pwd

/content/drive/My Drive/Colab Notebooks/全館暖冷房WG


'/content/drive/My Drive/Colab Notebooks/全館暖冷房WG'

In [ ]:
import pyhees.section4_2 as dc
import pyhees.section4_2_a as dc_a
import pyhees.section4_2_b as dc_spec

In [ ]:
import os
import sys

import numpy as np

from pyhees.section2_1_b import get_f_prim
from pyhees.section3_1 import get_Q
from pyhees.section3_2 import calc_r_env, get_Q_dash, get_eta_H, get_eta_C
from pyhees.section4_1 import calc_heating_load, calc_cooling_load, get_virtual_heating_devices, get_alpha_UT_H_A, calc_Q_UT_H_A_d_t#,\
#    calc_E_E_H_hs_A_d_t, calc_E_UT_H_d_t__modeA, calc_E_E_C_hs_d_t, calc_E_UT_C_d_t
from pyhees.section4_1_a import calc_heating_mode

In [ ]:
# ダクト式セントラル空調機
import pyhees.section4_2 as dc
import pyhees.section4_2_a as dc_a
import pyhees.section4_2_b as dc_spec

# エアーコンディショナー
import pyhees.section4_3 as rac
import pyhees.section4_3_a as rac_spec

from functools import lru_cache

import datetime

import pyhees.section3_1 as ld

from pyhees.section3_2_8 import \
    get_r_env

from pyhees.section11_1 import \
    load_outdoor, \
    get_T_ex, \
    get_Theta_ex, \
    get_X_ex, \
    calc_h_ex, \
    load_climate, \
    get_J

from pyhees.section3_1 import \
    get_A_NR

from pyhees.section4_7_i import \
    get_A_A_R

from pyhees.section11_3 import \
    load_schedule, \
    get_schedule_ac

In [ ]:
!pip install japanize-matplotlib                                                        
import japanize_matplotlib                                                              #日本語化matplotlibをimport     グラフ描画
import matplotlib.pyplot as plt                                                         #matplotlib.pyplotをimport      グラフ描画
import matplotlib.dates as mdates                                                       #matplotlib.pyplotをimport      グラフ描画
import matplotlib as mpl                                                                #matplotlibをimport             グラフ描画

from datetime import datetime, timedelta                                                #日付

     |████████████████████████████████| 4.1MB 2.8MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.2-cp36-none-any.whl size=4120191 sha256=716dd8868852f5436deddb4e71b8a9529eb67eaf5bc3352308138ca0807ab1d0
  Stored in directory: /root/.cache/pip/wheels/9c/f9/fc/bc052ce743a03f94ccc7fda73d1d389ce98216c6ffaaf65afc
Successfully built japanize-matplotlib


/usr/local/lib/python3.6/dist-packages/japanize_matplotlib/japanize_matplotlib.py:15: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  font_list = font_manager.createFontList(font_files)


In [ ]:
def calc_E_E_H_d_t(Theta_hs_out_d_t, Theta_hs_in_d_t, V_hs_supply_d_t, V_hs_vent_d_t, C_df_H_d_t,
           q_hs_rtd_H, V_hs_dsgn_H, P_hs_mid_H, P_hs_rtd_H, P_fan_rtd_H, P_fan_mid_H, q_hs_min_H, q_hs_mid_H,
           V_fan_rtd_H, V_fan_mid_H, EquipmentSpec, region):

    df_output = pd.DataFrame(index =pd.date_range('2019-01-01 01:00:00', '2020-01-01 00:00:00', freq='h'))

    # 外気条件
    outdoor = load_outdoor()
    Theta_ex_d_t = get_Theta_ex(region, outdoor)

    # (3)
    q_hs_H_d_t = dc_a.get_q_hs_H_d_t(Theta_hs_out_d_t, Theta_hs_in_d_t, V_hs_supply_d_t, C_df_H_d_t, region)
    df_output['q_hs_H_d_t'] = q_hs_H_d_t

    # (37)
    E_E_fan_H_d_t = dc_a.get_E_E_fan_H_d_t(P_fan_rtd_H, V_hs_vent_d_t, V_hs_supply_d_t, V_hs_dsgn_H, q_hs_H_d_t)
    df_output['E_E_fan_H_d_t'] = E_E_fan_H_d_t

    # (20)
    e_th_mid_H = dc_a.calc_e_th_mid_H(V_fan_mid_H, q_hs_mid_H)
    df_output['e_th_mid_H'] = e_th_mid_H

    # (19)
    e_th_rtd_H = dc_a.calc_e_th_rtd_H(V_fan_rtd_H, q_hs_rtd_H)
    df_output['e_th_rtd_H'] = e_th_rtd_H

    # (17)
    e_th_H_d_t = dc_a.calc_e_th_H_d_t(Theta_ex_d_t, Theta_hs_in_d_t, Theta_hs_out_d_t, V_hs_supply_d_t)
    df_output['e_th_H_d_t'] = e_th_H_d_t

    # (11)
    e_r_rtd_H = dc_a.get_e_r_rtd_H(e_th_rtd_H, q_hs_rtd_H, P_hs_rtd_H, P_fan_rtd_H)
    df_output['e_r_rtd_H'] = e_r_rtd_H

    # (15)
    e_r_min_H = dc_a.get_e_r_min_H(e_r_rtd_H)
    df_output['e_r_min_H'] = e_r_min_H

    # (13)
    e_r_mid_H = dc_a.get_e_r_mid_H(e_r_rtd_H, e_th_mid_H, q_hs_mid_H, P_hs_mid_H, P_fan_mid_H, EquipmentSpec)
    df_output['e_r_mid_H'] = e_r_mid_H

    # (9)
    e_r_H_d_t = dc_a.get_e_r_H_d_t(q_hs_H_d_t, q_hs_rtd_H, q_hs_min_H, q_hs_mid_H, e_r_mid_H, e_r_min_H, e_r_rtd_H)
    df_output['e_r_H_d_t'] = e_r_H_d_t

    # (7)
    e_hs_H_d_t = dc_a.get_e_hs_H_d_t(e_th_H_d_t, e_r_H_d_t)
    df_output['e_hs_H_d_t'] = e_hs_H_d_t

    # (5)
    E_E_comp_H_d_t = dc_a.get_E_E_comp_H_d_t(q_hs_H_d_t, e_hs_H_d_t)
    df_output['E_E_comp_H_d_t'] = E_E_comp_H_d_t

    # (1)
    E_E_H_d_t = E_E_comp_H_d_t + E_E_fan_H_d_t

    df_output.to_csv('/content/drive/My Drive/Colab Notebooks/全館暖冷房WG/output_CTRL_heating2.csv')
    return E_E_H_d_t


In [ ]:
def calc_E_E_H_hs_A_d_t(A_A, A_MR, A_OR, A_env, mu_H, mu_C, Q, H_A, L_H_d_t_i, L_CS_d_t_i, L_CL_d_t_i, region):
    """ 住戸全体を連続的に暖房する方式における暖房設備機器の消費電力量（kWh/h）を計算する

    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :param A_env: 外皮の部位の面積の合計 (m2)
    :type A_env: float
    :param mu_H: 当該住戸の暖房期の日射取得係数 ((W/m2)/(W/m2))
    :type mu_H: float
    :param mu_C: 当該住戸の冷房期の日射取得係数 ((W/m2)/(W/m2))
    :type mu_C: float
    :param Q: 当該住戸の熱損失係数 (W/m2K)
    :type Q: float
    :param H_A: 暖房方式
    :type H_A: dict
    :param L_H_d_t_i: 暖房区画i=1-5それぞれの暖房負荷
    :type L_H_d_t_i: ndarray
    :param L_CS_d_t_i: 冷房区画i=1-5それぞれの冷房顕熱負荷
    :type L_CS_d_t_i: ndarray
    :param L_CL_d_t_i: 冷房区画i=1-5それぞれの冷房潜熱負荷
    :type L_CL_d_t_i: ndarray
    :param region: 省エネルギー地域区分
    :type region: int
    :return: 住戸全体を連続的に暖房する方式における暖房設備機器の消費電力量（kWh/h）
    :rtype: ndarray
    """
    # VAV方式の採用
    if 'VAV' in H_A:
        VAV = H_A['VAV']

    # 全般換気機能の有無
    if 'general_ventilation' in H_A:
        general_ventilation = H_A['general_ventilation']

    # ダクトが通過する空間
    if 'duct_insulation' in H_A:
        duct_insulation = H_A['duct_insulation']

    # 機器の仕様の
    if H_A['EquipmentSpec'] == '入力しない':
        # 付録B
        EquipmentSpec = '入力しない'
        q_hs_rtd_H = dc_spec.get_q_hs_rtd_H(region, A_A)
        q_hs_mid_H = dc_spec.get_q_hs_mid_H(q_hs_rtd_H)
        q_hs_min_H = dc_spec.get_q_hs_min_H(q_hs_rtd_H)
        P_hs_rtd_H = dc_spec.get_P_hs_rtd_H(q_hs_rtd_H)
        V_fan_rtd_H = dc_spec.get_V_fan_rtd_H(q_hs_rtd_H)
        V_fan_mid_H = dc_spec. get_V_fan_mid_H(q_hs_mid_H)
        P_fan_rtd_H = dc_spec.get_P_fan_rtd_H(V_fan_rtd_H)
        P_fan_mid_H = dc_spec.get_P_fan_mid_H(V_fan_mid_H)
        P_hs_mid_H = np.NAN
    elif H_A['EquipmentSpec'] == '定格能力試験の値を入力する':
        EquipmentSpec = '定格能力試験の値を入力する'
        q_hs_rtd_H = H_A['q_hs_rtd_H']
        P_hs_rtd_H = H_A['P_hs_rtd_H']
        V_fan_rtd_H =H_A['V_fan_rtd_H']
        P_fan_rtd_H = H_A['P_fan_rtd_H']
        q_hs_mid_H = dc_spec.get_q_hs_mid_H(q_hs_rtd_H)
        q_hs_min_H = dc_spec.get_q_hs_min_H(q_hs_rtd_H)
        V_fan_mid_H = dc_spec. get_V_fan_mid_H(q_hs_mid_H)
        P_fan_mid_H = dc_spec.get_P_fan_mid_H(V_fan_mid_H)
        P_hs_mid_H = np.NAN
    elif H_A['EquipmentSpec'] == '定格能力試験と中間能力試験の値を入力する':
        EquipmentSpec = '定格能力試験と中間能力試験の値を入力する'
        q_hs_rtd_H = H_A['q_hs_rtd_H']
        P_hs_rtd_H = H_A['P_hs_rtd_H']
        V_fan_rtd_H = H_A['V_fan_rtd_H']
        P_fan_rtd_H = H_A['P_fan_rtd_H']
        q_hs_mid_H = H_A['q_hs_mid_H']
        P_hs_mid_H = H_A['P_hs_mid_H']
        V_fan_mid_H = H_A['V_fan_mid_H']
        P_fan_mid_H = H_A['P_fan_mid_H']
        q_hs_min_H = dc_spec.get_q_hs_min_H(q_hs_rtd_H)
    else:
        raise ValueError(H_A['EquipmentSpec'])

     # 設計風量(暖房)
    if 'V_hs_dsgn_H' in H_A:
        V_hs_dsgn_H = H_A['V_hs_dsgn_H']
    else:
        V_hs_dsgn_H = dc_spec.get_V_fan_dsgn_H(V_fan_rtd_H)

    # 定格冷房能力
    q_hs_rtd_C = None

    # 設計風量(冷房)
    V_hs_dsgn_C = None

    print('【暖房消費電力の計算】')
    print('\t床面積の合計 (m2)\t\t\t\t\t\t\tA_A = {}'.format(A_A))
    print('\t主たる居室の床面積 (m2)\t\t\t\t\t\t\tA_MR = {}'.format(A_MR))
    print('\tその他の居室の床面積 (m2)\t\t\t\t\t\tA_OR = {}'.format(A_OR))
    print('\t外皮の部位の面積の合計 (m2)\t\t\t\t\t\tA_env = {}'.format(A_env))
    print('\t当該住戸の暖房期の日射取得係数 ((W/m2)/(W/m2))\t\t\t\tmu_H = {}'.format(mu_H))
    print('\t当該住戸の冷房期の日射取得係数 ((W/m2)/(W/m2))\t\t\t\tmu_C = {}'.format(mu_C))
    print('\t当該住戸の熱損失係数 (W/m2K)\t\t\t\t\t\tQ = {}'.format(Q))
    print('\t暖房方式\t\t\t\t\t\t\t\tH_A = {}'.format(H_A))
    print('\t暖房負荷\t\t\t\t\t\t\t\tL_H_d_t_i')
    print('\t冷房顕熱負荷\t\t\t\t\t\t\t\tL_CS_d_t_i')
    print('\t冷房潜熱負荷\t\t\t\t\t\t\t\tL_CL_d_t_i')
    print('\t省エネルギー地域区分\t\t\t\t\t\t\tregion = {}'.format(region))
    print('\t定格暖房能力 (W)\t\t\t\t\t\t\tq_hs_rtd_H = {}'.format(q_hs_rtd_H))
    print('\t中間暖房能力 (W)\t\t\t\t\t\t\tq_hs_mid_H = {}'.format(q_hs_mid_H))
    print('\t最小暖房能力 (W)\t\t\t\t\t\t\tq_hs_min_H = {}'.format(q_hs_min_H))
    print('\t定格暖房能力運転時の送風機の風量(m3/h)\t\t\t\t\tV_fan_rtd_H = {}'.format(V_fan_rtd_H))
    print('\t暖房時の送風機の設計風量(m3/h)\t\t\t\t\t\tV_hs_dsgn_H = {}'.format(V_hs_dsgn_H))
    print('\t定格暖房能力運転時の送風機の消費電力(W)\t\t\t\t\tP_fan_rtd_H = {}'.format(P_fan_rtd_H))

    _, _, _, _, Theta_hs_out_d_t, Theta_hs_in_d_t, \
    _, _, V_hs_supply_d_t, V_hs_vent_d_t, C_df_H_d_t = dc.calc_Q_UT_A(A_A, A_MR, A_OR, A_env, mu_H, mu_C,
                                                                    q_hs_rtd_H, q_hs_rtd_C, V_hs_dsgn_H,
                                                                    V_hs_dsgn_C, Q, VAV, general_ventilation,
                                                                    duct_insulation, region, L_H_d_t_i,
                                                                    L_CS_d_t_i, L_CL_d_t_i)

    # 電力消費量の計算
    E_E_H_d_t = calc_E_E_H_d_t(
        Theta_hs_out_d_t=Theta_hs_out_d_t,
        Theta_hs_in_d_t=Theta_hs_in_d_t,
        V_hs_supply_d_t=V_hs_supply_d_t,
        V_hs_vent_d_t=V_hs_vent_d_t,
        C_df_H_d_t=C_df_H_d_t,
        q_hs_rtd_H=q_hs_rtd_H,
        V_hs_dsgn_H=V_hs_dsgn_H,
        P_hs_mid_H=P_hs_mid_H,
        P_hs_rtd_H=P_hs_rtd_H,
        P_fan_rtd_H=P_fan_rtd_H,
        P_fan_mid_H=P_fan_mid_H,
        q_hs_min_H=q_hs_min_H,
        q_hs_mid_H=q_hs_mid_H,
        V_fan_rtd_H=V_fan_rtd_H,
        V_fan_mid_H=V_fan_mid_H,
        EquipmentSpec=EquipmentSpec,
        region=region
    )

    df_output = pd.DataFrame(index =pd.date_range('2019-01-01 01:00:00', '2020-01-01 00:00:00', freq='h'))
    df_output['Theta_hs_out_d_t'] = Theta_hs_out_d_t
    df_output['Theta_hs_in_d_t'] = Theta_hs_in_d_t
    df_output['V_hs_supply_d_t'] = V_hs_supply_d_t
    df_output['V_hs_vent_d_t'] = V_hs_vent_d_t
    df_output['C_df_H_d_t'] = C_df_H_d_t
    df_output['q_hs_rtd_H'] = q_hs_rtd_H
    df_output['V_hs_dsgn_H'] = V_hs_dsgn_H
    df_output['P_hs_mid_H'] = P_hs_mid_H
    df_output['P_hs_rtd_H'] = P_hs_rtd_H
    df_output['P_fan_rtd_H'] = P_fan_rtd_H
    df_output['P_fan_mid_H'] = P_fan_mid_H
    df_output['q_hs_min_H'] = q_hs_min_H
    df_output['q_hs_mid_H'] = q_hs_mid_H
    df_output['V_fan_rtd_H'] = V_fan_rtd_H
    df_output['V_fan_mid_H'] = V_fan_mid_H
    df_output['Equipmentspec'] = EquipmentSpec
    df_output['region'] = region
    df_output.to_csv('/content/drive/My Drive/Colab Notebooks/全館暖冷房WG/output_CTRL_heating1.csv')

    return E_E_H_d_t

In [ ]:
def get_E_E_C_d_t(Theta_hs_out_d_t, Theta_hs_in_d_t,  X_hs_out_d_t, X_hs_in_d_t, V_hs_supply_d_t, V_hs_vent_d_t,
                  q_hs_rtd_C, V_hs_dsgn_C, q_hs_mid_C, q_hs_min_C,
                  P_fan_rtd_C, P_fan_mid_C, P_hs_rtd_C, P_hs_mid_C, V_fan_rtd_C, V_fan_mid_C, EquipmentSpec, region):

    # 外気条件
    outdoor = load_outdoor()
    Theta_ex_d_t = get_Theta_ex(region, outdoor)
    df_output = pd.DataFrame(index =pd.date_range('2019-01-01 01:00:00', '2020-01-01 00:00:00', freq='h'))

    # (4)
    q_hs_C_d_t = dc_a.get_q_hs_C_d_t(Theta_hs_out_d_t, Theta_hs_in_d_t, X_hs_out_d_t, X_hs_in_d_t, V_hs_supply_d_t, region)
    df_output['q_hs_C_d_t'] = q_hs_C_d_t

    # (38)
    E_E_fan_C_d_t = dc_a.get_E_E_fan_C_d_t(P_fan_rtd_C, V_hs_vent_d_t, V_hs_supply_d_t, V_hs_dsgn_C, q_hs_C_d_t)
    df_output['E_E_fan_C_d_t'] = E_E_fan_C_d_t

    # (22)
    e_th_mid_C = dc_a.calc_e_th_mid_C(V_fan_mid_C, q_hs_mid_C)
    df_output['e_th_mid_C'] = e_th_mid_C

    # (21)
    e_th_rtd_C = dc_a.calc_e_th_rtd_C(V_fan_rtd_C, q_hs_rtd_C)
    df_output['e_th_rtd_C'] = e_th_rtd_C

    # (18)
    e_th_C_d_t = dc_a.calc_e_th_C_d_t(Theta_ex_d_t, Theta_hs_in_d_t, X_hs_in_d_t, Theta_hs_out_d_t, V_hs_supply_d_t)
    df_output['e_th_C_d_t'] = e_th_C_d_t

    # (12)
    e_r_rtd_C = dc_a.get_e_r_rtd_C(e_th_rtd_C, q_hs_rtd_C, P_hs_rtd_C, P_fan_rtd_C)
    df_output['e_r_rtd_C'] = e_r_rtd_C

    # (16)
    e_r_min_C = dc_a.get_e_r_min_C(e_r_rtd_C)
    df_output['e_r_min_C'] = e_r_min_C

    # (14)
    e_r_mid_C = dc_a.get_e_r_mid_C(e_r_rtd_C, e_th_mid_C, q_hs_mid_C, P_hs_mid_C, P_fan_mid_C, EquipmentSpec)
    df_output['e_r_mid_C'] = e_r_mid_C

    # (10)
    e_r_C_d_t = dc_a.get_e_r_C_d_t(q_hs_C_d_t, q_hs_rtd_C, q_hs_min_C, q_hs_mid_C, e_r_mid_C, e_r_min_C, e_r_rtd_C)
    df_output['e_r_C_d_t'] = e_r_C_d_t

    # (8)
    e_hs_C_d_t = dc_a.get_e_hs_C_d_t(e_th_C_d_t, e_r_C_d_t)
    df_output['e_hs_C_d_t'] = e_hs_C_d_t

    # (6)
    E_E_comp_C_d_t = dc_a.get_E_E_comp_C_d_t(q_hs_C_d_t, e_hs_C_d_t)
    df_output['E_E_comp_C_d_t'] = E_E_comp_C_d_t

    # (2)
    E_E_C_d_t = E_E_comp_C_d_t + E_E_fan_C_d_t

    df_output.to_csv('/content/drive/My Drive/Colab Notebooks/全館暖冷房WG/output_CTRL_cooling2.csv')
    return E_E_C_d_t


In [ ]:
def calc_E_E_C_hs_d_t(region, A_A, A_MR, A_OR, A_env, mu_H, mu_C, Q, C_A, C_MR, C_OR, L_H_d_t, L_CS_d_t, L_CL_d_t):
    """　冷房設備機器の消費電力量（kWh/h）(22a, 23a)を取得する

    :param region: 省エネルギー地域区分
    :type region: int
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :param A_env: 外皮の部位の面積の合計 (m2)
    :type A_env: float
    :param mu_H: 当該住戸の暖房期の日射取得係数 ((W/m2)/(W/m2))
    :type mu_H: float
    :param mu_C: 当該住戸の冷房期の日射取得係数 ((W/m2)/(W/m2))
    :type mu_C: float
    :param Q: 当該住戸の熱損失係数 (W/m2K)
    :type Q: float
    :param C_A: 冷房方式
    :type C_A: dict
    :param C_MR: 主たる居室の冷房機器
    :type C_MR: dict
    :param C_OR: その他の居室の冷房機器
    :type C_OR: dict
    :param L_H_d_t: 暖房区画の暖房負荷
    :type L_H_d_t: ndarray
    :param L_CS_d_t: 冷房区画の冷房顕熱負荷
    :type L_CS_d_t: ndarray
    :param L_CL_d_t: 冷房区画の冷房潜熱負荷
    :type L_CL_d_t: ndarray
    :return: 冷房設備機器の消費電力量（kWh/h）
    :rtype: ndarray
    """
    if C_A is not None:
        # VAV方式の採用
        if 'VAV' in C_A:
            VAV = C_A['VAV']

        # 全般換気機能の有無
        if 'general_ventilation' in C_A:
            general_ventilation = C_A['general_ventilation']

        # ダクトが通過する空間
        if 'duct_insulation' in C_A:
            duct_insulation = C_A['duct_insulation']

        # 機器の仕様の
        if C_A['EquipmentSpec'] == '入力しない':
            # 付録B
            EquipmentSpec = '入力しない'
            q_hs_rtd_C = dc_spec.get_q_hs_rtd_C(region, A_A)
            q_hs_mid_C = dc_spec.get_q_hs_mid_C(q_hs_rtd_C)
            q_hs_min_C = dc_spec.get_q_hs_min_C(q_hs_rtd_C)
            P_hs_rtd_C = dc_spec.get_P_hs_rtd_C(q_hs_rtd_C)
            V_fan_rtd_C = dc_spec.get_V_fan_rtd_C(q_hs_rtd_C)
            V_fan_mid_C = dc_spec.get_V_fan_mid_C(q_hs_mid_C)
            P_fan_rtd_C = dc_spec.get_P_fan_rtd_C(V_fan_rtd_C)
            P_fan_mid_C = dc_spec.get_P_fan_mid_C(V_fan_mid_C)
            P_hs_mid_C = np.NAN
        elif C_A['EquipmentSpec'] == '定格能力試験の値を入力する':
            EquipmentSpec = '定格能力試験の値を入力する'
            q_hs_rtd_C = C_A['q_hs_rtd_C']
            P_hs_rtd_C = C_A['P_hs_rtd_C']
            V_fan_rtd_C = C_A['V_fan_rtd_C']
            P_fan_rtd_C = C_A['P_fan_rtd_C']
            q_hs_mid_C = dc_spec.get_q_hs_mid_C(q_hs_rtd_C)
            q_hs_min_C = dc_spec.get_q_hs_min_C(q_hs_rtd_C)
            V_fan_mid_C = dc_spec.get_V_fan_mid_C(q_hs_mid_C)
            P_fan_mid_C = dc_spec.get_P_fan_mid_C(V_fan_mid_C)
            P_hs_mid_C = np.NAN
        elif C_A['EquipmentSpec'] == '定格能力試験と中間能力試験の値を入力する':
            EquipmentSpec = '定格能力試験と中間能力試験の値を入力する'
            q_hs_rtd_C = C_A['q_hs_rtd_C']
            P_hs_rtd_C = C_A['P_hs_rtd_C']
            V_fan_rtd_C = C_A['V_fan_rtd_C']
            P_fan_rtd_C = C_A['P_fan_rtd_C']
            q_hs_mid_C = C_A['q_hs_mid_C']
            P_hs_mid_C = C_A['P_hs_mid_C']
            V_fan_mid_C = C_A['V_fan_mid_C']
            P_fan_mid_C = C_A['P_fan_mid_C']
            q_hs_min_C = dc_spec.get_q_hs_min_C(q_hs_rtd_C)
        else:
            raise ValueError(C_A['EquipmentSpec'])

        # 設計風量(冷房)
        if 'V_hs_dsgn_C' in C_A:
            V_hs_dsgn_C = C_A['V_hs_dsgn_C']
        else:
            V_hs_dsgn_C = dc_spec.get_V_fan_dsgn_C(V_fan_rtd_C)

        # 定格暖房能力
        q_hs_rtd_H = None

        # 設計風量(暖房)
        V_hs_dsgn_H = None

        print('【冷房消費電力の計算】')
        print('\t床面積の合計 (m2)\t\t\t\t\t\t\tA_A = {}'.format(A_A))
        print('\t主たる居室の床面積 (m2)\t\t\t\t\t\t\tA_MR = {}'.format(A_MR))
        print('\tその他の居室の床面積 (m2)\t\t\t\t\t\tA_OR = {}'.format(A_OR))
        print('\t外皮の部位の面積の合計 (m2)\t\t\t\t\t\tA_env = {}'.format(A_env))
        print('\t当該住戸の暖房期の日射取得係数 ((W/m2)/(W/m2))\t\t\t\tmu_H = {}'.format(mu_H))
        print('\t当該住戸の冷房期の日射取得係数 ((W/m2)/(W/m2))\t\t\t\tmu_C = {}'.format(mu_C))
        print('\t当該住戸の熱損失係数 (W/m2K)\t\t\t\t\t\tQ = {}'.format(Q))
        print('\t冷房方式\t\t\t\t\t\t\t\tH_A = {}'.format(C_A))
        print('\t暖房負荷\t\t\t\t\t\t\t\tL_H_d_t_i')
        print('\t冷房顕熱負荷\t\t\t\t\t\t\t\tL_CS_d_t_i')
        print('\t冷房潜熱負荷\t\t\t\t\t\t\t\tL_CL_d_t_i')
        print('\t省エネルギー地域区分\t\t\t\t\t\t\tregion = {}'.format(region))
        print('\t定格冷房能力 (W)\t\t\t\t\t\t\tq_hs_rtd_C = {}'.format(q_hs_rtd_C))
        print('\t中間冷房能力 (W)\t\t\t\t\t\t\tq_hs_mid_C = {}'.format(q_hs_mid_C))
        print('\t最小冷房能力 (W)\t\t\t\t\t\t\tq_hs_min_C = {}'.format(q_hs_min_C))
        print('\t定格冷房能力運転時の送風機の風量(m3/h)\t\t\t\t\tV_fan_rtd_C = {}'.format(V_fan_rtd_C))
        print('\t冷房時の送風機の設計風量(m3/h)\t\t\t\t\t\tV_hs_dsgn_C = {}'.format(V_hs_dsgn_C))
        print('\t定格冷房能力運転時の送風機の消費電力(W)\t\t\t\t\tP_fan_rtd_C = {}'.format(P_fan_rtd_C))

        _, _, _, _, \
        Theta_hs_out_d_t, Theta_hs_in_d_t, X_hs_out_d_t, \
        X_hs_in_d_t, V_hs_supply_d_t, V_hs_vent_d_t, _ = dc.calc_Q_UT_A(A_A, A_MR, A_OR, A_env, mu_H, mu_C, q_hs_rtd_H,
                                                                       q_hs_rtd_C, V_hs_dsgn_H, V_hs_dsgn_C, Q, VAV,
                                                                       general_ventilation, duct_insulation, region,
                                                                       L_H_d_t, L_CS_d_t, L_CL_d_t)

        E_E_C_d_t_i = get_E_E_C_d_t(
            Theta_hs_out_d_t=Theta_hs_out_d_t,
            Theta_hs_in_d_t=Theta_hs_in_d_t,
            X_hs_out_d_t=X_hs_out_d_t,
            X_hs_in_d_t=X_hs_in_d_t,
            V_hs_supply_d_t=V_hs_supply_d_t,
            V_hs_vent_d_t=V_hs_vent_d_t,
            q_hs_rtd_C=q_hs_rtd_C,
            V_hs_dsgn_C=V_hs_dsgn_C,
            q_hs_mid_C=q_hs_mid_C,
            q_hs_min_C=q_hs_min_C,
            P_fan_rtd_C=P_fan_rtd_C,
            P_fan_mid_C=P_fan_mid_C,
            P_hs_rtd_C=P_hs_rtd_C,
            P_hs_mid_C=P_hs_mid_C,
            V_fan_rtd_C=V_fan_rtd_C,
            V_fan_mid_C=V_fan_mid_C,
            EquipmentSpec=EquipmentSpec,
            region=region
        )

        df_output = pd.DataFrame(index =pd.date_range('2019-01-01 01:00:00', '2020-01-01 00:00:00', freq='h'))
        df_output['Theta_hs_out_d_t'] = Theta_hs_out_d_t
        df_output['Theta_hs_in_d_t'] = Theta_hs_in_d_t
        df_output['X_hs_out_d_t'] = X_hs_out_d_t
        df_output['X_hs_in_d_t'] = X_hs_in_d_t
        df_output['V_hs_supply_d_t'] = V_hs_supply_d_t
        df_output['V_hs_vent_d_t'] = V_hs_vent_d_t
        df_output['q_hs_rtd_C'] = q_hs_rtd_C
        df_output['V_hs_dsgn_C'] = V_hs_dsgn_C
        df_output['q_hs_mid_C'] = q_hs_mid_C
        df_output['q_hs_min_C'] = q_hs_min_C
        df_output['P_fan_rtd_C'] = P_fan_rtd_C
        df_output['P_fan_mid_C'] = P_fan_mid_C
        df_output['P_hs_rtd_C'] = P_hs_rtd_C
        df_output['P_hs_mid_C'] = P_hs_mid_C
        df_output['V_fan_rtd_C'] = V_fan_rtd_C
        df_output['V_fan_mid_C'] = V_fan_mid_C
        df_output['EquipmentSpec'] = EquipmentSpec
        df_output['region'] = region
        df_output.to_csv('/content/drive/My Drive/Colab Notebooks/全館暖冷房WG/output_CTRL_cooling1.csv')

        return E_E_C_d_t_i
    elif C_MR is not None or C_OR is not None:
        # 消費電力量の計算 (23a)
        return calc_E_E_C_hs_MR_d_t(region, A_A, A_MR, A_OR, C_MR, L_CS_d_t, L_CL_d_t) \
               + calc_E_E_C_hs_OR_d_t(region, A_A, A_MR, A_OR, C_OR, L_CS_d_t, L_CL_d_t)
    else:
        return np.zeros(24*365)

In [ ]:
def calc_E_UT_H_d_t__modeA(H_A, A_A, A_MR, A_OR, A_env, mu_H, mu_C, Q, region, L_H_d_t_i, L_CS_d_t_i, L_CL_d_t_i):
    """ 暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値（MJ/h）(13)を取得する

    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :param A_env: 外皮の部位の面積の合計 (m2)
    :type A_env: float
    :param mu_H: 当該住戸の暖房期の日射取得係数 ((W/m2)/(W/m2))
    :type mu_H: float
    :param mu_C: 当該住戸の冷房期の日射取得係数 ((W/m2)/(W/m2))
    :type mu_C: float
    :param Q: 当該住戸の熱損失係数 (W/m2K)
    :type Q: float
    :param region: 省エネルギー地域区分
    :type region: int
    :param L_H_d_t_i: 暖房区画の暖房負荷
    :type L_H_d_t_i: ndarray
    :param L_CS_d_t_i: 冷房区画の冷房顕熱負荷
    :type L_CS_d_t_i: ndarray
    :param L_CL_d_t_i: 冷房区画の冷房潜熱負荷
    :type L_CL_d_t_i: ndarray
    :return: 暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値（MJ/h）
    :rtype: ndarray
    """
    # 未処理暖房負荷を未処理暖房負荷の設計一次エネルギー消費量相当値に換算するための係数
    alpha_UT_H_A = get_alpha_UT_H_A(region)

    # VAV方式の採用
    if 'VAV' in H_A:
        VAV = H_A['VAV']

    # 全般換気機能の有無
    if 'general_ventilation' in H_A:
        general_ventilation = H_A['general_ventilation']

    # ダクトが通過する空間
    if 'duct_insulation' in H_A:
        duct_insulation = H_A['duct_insulation']

    # 定格暖房能力
    if 'q_hs_rtd_H' in H_A:
        q_hs_rtd_H = H_A['q_hs_rtd_H']
    else:
        q_hs_rtd_H = dc_spec.get_q_hs_rtd_H(region, A_A)

    # 定格風量(暖房)
    if 'V_fan_rtd_H' in H_A:
        V_fan_rtd_H = H_A['V_fan_rtd_H']
    else:
        V_fan_rtd_H = dc_spec.get_V_fan_rtd_H(q_hs_rtd_H)

    # 設計風量(暖房)
    if 'V_hs_dsgn_H' in H_A:
        V_hs_dsgn_H = H_A['V_hs_dsgn_H']
    else:
        V_hs_dsgn_H = dc_spec.get_V_fan_dsgn_H(V_fan_rtd_H)

    # 定格冷房能力
    q_hs_rtd_C = None

    # 設計風量(冷房)
    V_hs_dsgn_C = None

    # 未処理負荷を取得
    Q_UT_H_A_d_t = calc_Q_UT_H_A_d_t(A_A, A_MR, A_OR, A_env, mu_H, mu_C, q_hs_rtd_H, q_hs_rtd_C, V_hs_dsgn_H, V_hs_dsgn_C, Q,
                     VAV, general_ventilation, duct_insulation, region, L_H_d_t_i, L_CS_d_t_i, L_CL_d_t_i)

    return Q_UT_H_A_d_t * alpha_UT_H_A


In [ ]:
def calc_E_UT_C_d_t(region, A_A, A_MR, A_OR, A_env, mu_H, mu_C, Q, C_A, C_MR, C_OR, L_H_d_t, L_CS_d_t, L_CL_d_t, mode_C):
    """　冷房設備の未処理暖房負荷の設計一次エネルギー消費量相当値（MJ/h）を計算する

    :param region: 省エネルギー地域区分
    :type region: int
    :param A_A: 床面積の合計 (m2)
    :type A_A: float
    :param A_MR: 主たる居室の床面積 (m2)
    :type A_MR: float
    :param A_OR: その他の居室の床面積 (m2)
    :type A_OR: float
    :param A_env: 外皮の部位の面積の合計 (m2)
    :type A_env: float
    :param mu_H: 当該住戸の暖房期の日射取得係数 ((W/m2)/(W/m2))
    :type mu_H: float
    :param mu_C: 当該住戸の冷房期の日射取得係数 ((W/m2)/(W/m2))
    :type mu_C: float
    :param Q: 当該住戸の熱損失係数 (W/m2K)
    :type Q: float
    :param C_A: 冷房方式
    :type C_A: dict
    :param C_MR: 主たる居室の冷房機器
    :type C_MR: dict
    :param C_OR: その他の居室の冷房機器
    :type C_OR: dict
    :param L_CS_d_t: 冷房区画の冷房顕熱負荷
    :type L_CS_d_t: ndarray
    :param L_CL_d_t: 冷房区画の冷房潜熱負荷
    :type L_CL_d_t: ndarray
    :param mode_C: 冷房方式
    :type mode_C: str
    :return: 冷房設備の未処理暖房負荷の設計一次エネルギー消費量相当値（MJ/h）
    :rtype: ndarray
    """

    if mode_C == '住戸全体を連続的に冷房する方式':
        # VAV方式の採用
        if 'VAV' in C_A:
            VAV = C_A['VAV']

        # 全般換気機能の有無
        if 'general_ventilation' in C_A:
            general_ventilation = C_A['general_ventilation']

        # ダクトが通過する空間
        if 'duct_insulation' in C_A:
            duct_insulation = C_A['duct_insulation']

        # 機器の仕様の
        if C_A['EquipmentSpec'] == '入力しない':
            # 付録B
            q_hs_rtd_C = dc_spec.get_q_hs_rtd_C(region, A_A)
        elif C_A['EquipmentSpec'] == '定格能力試験の値を入力する':
            q_hs_rtd_C = C_A['q_hs_rtd_C']
        elif C_A['EquipmentSpec'] == '定格能力試験と中間能力試験の値を入力する':
            q_hs_rtd_C = C_A['q_hs_rtd_C']
        else:
            raise ValueError(C_A['EquipmentSpec'])

        # 定格風量（冷房）
        if 'V_fan_rtd_C' in C_A:
            V_fan_rtd_C = C_A['V_fan_rtd_C']
        else:
            V_fan_rtd_C = dc_spec.get_V_fan_rtd_H(q_hs_rtd_C)

        # 設計風量(冷房)
        if 'V_hs_dsgn_C' in C_A:
            V_hs_dsgn_C = C_A['V_hs_dsgn_C']
        else:
            V_hs_dsgn_C = dc_spec.get_V_fan_dsgn_C(V_fan_rtd_C)

        # 定格暖房能力
        q_hs_rtd_H = None

        # 設計風量(暖房)
        V_hs_dsgn_H = None

        E_UT_C_d_t, _, _, _, _, _, \
        _, _, _, _, _ = dc.calc_Q_UT_A(A_A, A_MR, A_OR, A_env, mu_H, mu_C, q_hs_rtd_H, q_hs_rtd_C, V_hs_dsgn_H, V_hs_dsgn_C, Q,
             VAV, general_ventilation, duct_insulation, region, L_H_d_t, L_CS_d_t, L_CL_d_t)

        E_UT_C_d_t = E_UT_C_d_t

    else:
        E_UT_C_d_t = np.zeros(24 * 365)


    return E_UT_C_d_t

In [ ]:
# ============================================================================
# 暖房および冷房機器にダクト式セントラル空調機を用いた際の
# 暖房および冷房設計一次エネルギー量を求めるサンプルコード
# ============================================================================

def get_basic():
    """基本情報の設定

    :return: 住宅タイプ、住宅建て方、床面積、地域区分、年間日射地域区分
    """
    # 住宅タイプ
    type = '一般住宅'

    # 住宅建て方
    tatekata = '戸建住宅'

    # 床面積
    A_A = 120.08
    A_MR = 29.81
    A_OR = 51.34

    # 地域区分
    region = 6

    # 年間日射地域区分
    sol_region = None

    return type, tatekata, A_A, A_MR, A_OR, region, sol_region


def get_env():
    """外皮の設定

    :return: 外皮条件
    """
    ENV = {
        'method': '当該住宅の外皮面積の合計を用いて評価する',
        'A_env': 307.51,
        'A_A': 120.08,
        'U_A': 0.87,
        'eta_A_H': 4.3,
        'eta_A_C': 2.8
    }

    # 自然風の利用 主たる居室
    NV_MR = 0
    # 自然風の利用 その他居室
    NV_OR = 0

    # 蓄熱
    TS = False

    # 床下空間を経由して外気を導入する換気方式の利用
    r_A_ufvnt = None

    # 床下空間の断熱
    underfloor_insulation = None

    return ENV, NV_MR, NV_OR, TS, r_A_ufvnt, underfloor_insulation


def get_heating():
    """暖房の設定

    :return: 暖房方式、住戸全体の暖房条件、主たる居室の暖房機器、その他居室の暖房機器、温水暖房の種類
    """
    # 暖房方式
    mode_H = '住戸全体を連続的に暖房する方式'

    # 住戸全体を暖房する
    H_A = {
        'type': 'ダクト式セントラル空調機',
        'duct_insulation': '全てもしくは一部が断熱区画外である',
        'VAV': False,
        'general_ventilation': True,
        'EquipmentSpec': '入力しない'
    }

    # 主たる居室暖房機器
    H_MR = None

    # その他居室暖房機器
    H_OR = None

    # 温水暖房機の種類
    H_HS = None

    return mode_H, H_A, H_MR, H_OR, H_HS


def get_cooling():
    """冷房の設定

    :return: 冷房方式、住戸全体の冷房条件、主たる居室冷房条件、その他居室冷房条件
    """
    # 冷房方式
    mode_C = '住戸全体を連続的に冷房する方式'

    # 住戸全体を冷房する
    C_A = {
        'type': 'ダクト式セントラル空調機',
        'duct_insulation': '全てもしくは一部が断熱区画外である',
        'VAV': False,
        'general_ventilation': True,
        'EquipmentSpec': '入力しない'
    }

    # 主たる居室冷房機器
    C_MR = None

    # その他居室冷房機器
    C_OR = None

    return mode_C, C_A, C_MR, C_OR


def get_heatexchangeventilation():
    """熱交換型換気の設定

    :return: 熱交換型換気
    """
    # 熱交換型換気
    HEX = None

    return HEX


def get_solarheat():
    """太陽熱利用の設定

    :return: 太陽熱利用
    """
    # 太陽熱利用
    SHC = None

    return SHC


def calc():
    """暖房設計一次エネルギー消費量

    :return: 各パラメータの計算結果
    """

    # ---------- 計算条件の取得 ----------

    # 基本情報を取得
    type, tatekata, A_A, A_MR, A_OR, region, sol_region = get_basic()

    # 外皮条件を取得
    ENV, NV_MR, NV_OR, TS, r_A_ufvnt, underfloor_insulation = get_env()

    # 暖房条件の取得
    mode_H, H_A, H_MR, H_OR, H_HS = get_heating()

    # 冷房条件の取得
    mode_C, C_A, C_MR, C_OR = get_cooling()

    # 熱交換型換気の取得
    HEX = get_heatexchangeventilation()

    # 太陽熱利用の取得
    SHC = get_solarheat()

    # ---------- その他計算条件を取得 ----------

    # 床面積の合計に対する外皮の部位の面積の合計の比
    r_env = calc_r_env(
        method='当該住戸の外皮の部位の面積等を用いて外皮性能を評価する方法',
        A_env=ENV['A_env'],
        A_A=A_A
    )

    # 熱損失係数（換気による熱損失を含まない）
    Q_dash = get_Q_dash(ENV['U_A'], r_env)
    # 熱損失係数
    Q = get_Q(Q_dash)

    # 日射取得係数の取得
    mu_H = get_eta_H(ENV['eta_A_H'], r_env)
    mu_C = get_eta_C( ENV['eta_A_C'], r_env)

    # 実質的な暖房機器の仕様を取得
    spec_MR, spec_OR = get_virtual_heating_devices(region, H_MR, H_OR)

    # 暖房方式及び運転方法の区分
    mode_MR, mode_OR = calc_heating_mode(region=region, H_MR=spec_MR, H_OR=spec_OR)

    # ---------- 暖房負荷の取得 ----------

    # 暖房負荷の取得
    L_H_d_t_i, _ = calc_heating_load(region, sol_region, A_A, A_MR, A_OR, Q, mu_H, mu_C, NV_MR, NV_OR, TS, r_A_ufvnt,
                                     HEX, underfloor_insulation, mode_H, mode_C, spec_MR, spec_OR, mode_MR, mode_OR,
                                     SHC)

    # ---------- 冷房負荷の取得 ----------

    # 冷房負荷の取得
    L_CS_d_t_i, L_CL_d_t_i = calc_cooling_load(region, A_A, A_MR, A_OR, Q, mu_H, mu_C, NV_MR, NV_OR, r_A_ufvnt,
                                               underfloor_insulation, mode_C, mode_H, mode_MR, mode_OR, TS, HEX)

    # ---------- 計算開始 ----------

    # 暖房設備機器の消費電力量（kWh/h）
    E_E_H_d_t = calc_E_E_H_hs_A_d_t(A_A, A_MR, A_OR, ENV['A_env'], mu_H, mu_C, Q, H_A, L_H_d_t_i,
                                    L_CS_d_t_i, L_CL_d_t_i, region)

    # 冷房設備機器の消費電力量（kWh/h）
    E_E_C_d_t = calc_E_E_C_hs_d_t(region, A_A, A_MR, A_OR, ENV['A_env'], mu_H, mu_C, Q, C_A, C_MR, C_OR,
                                  L_H_d_t_i, L_CS_d_t_i, L_CL_d_t_i)

    # 暖房設備の未処理暖房負荷の設計一次エネルギー消費量相当値（MJ/h）
    E_UT_H_d_t = calc_E_UT_H_d_t__modeA(H_A, A_A, A_MR, A_OR, ENV['A_env'], mu_H, mu_C, Q, region,
                                        L_H_d_t_i, L_CS_d_t_i, L_CL_d_t_i)

    # 冷房設備の未処理暖房負荷の設計一次エネルギー消費量相当値（MJ/h）
    E_UT_C_d_t = calc_E_UT_C_d_t(region, A_A, A_MR, A_OR, ENV['A_env'], mu_H, mu_C, Q, C_A, C_MR, C_OR,
                                 L_H_d_t_i, L_CS_d_t_i, L_CL_d_t_i, mode_C)

    df_output = pd.DataFrame(index =pd.date_range('2019-01-01 01:00:00', '2020-01-01 00:00:00', freq='h'))
    df_output['E_UT_H_d_t'] = E_UT_H_d_t
    df_output['E_UT_C_d_t'] = E_UT_C_d_t
    df_output.to_csv('/content/drive/My Drive/Colab Notebooks/全館暖冷房WG/output2_CTRL.csv')

    # 電気の量 1kWh を熱量に換算する係数
    f_prim = get_f_prim()

    # 1 時間当たりの暖房設備の設計一次エネルギー消費量（MJ/h）
    E_H_d_t = E_E_H_d_t * f_prim / 1000 + E_UT_H_d_t

    # 1 時間当たりの冷房設備の設計一次エネルギー消費量（MJ/h）
    E_C_d_t = E_E_C_d_t * f_prim / 1000 + E_UT_C_d_t

    # 1 年当たりの暖房設備の設計一次エネルギー消費量（MJ/年）
    E_H = np.sum(E_H_d_t)

    # 1 年当たりの冷房設備の設計一次エネルギー消費量（MJ/年）
    E_C = np.sum(E_C_d_t)

    print({'E_H': E_H, 'E_C': E_C})


In [ ]:
calc()

data/3-1_HukaData_151019_unifyLDK/IH6x_11_05_212_1SS.csv
data/3-1_HukaData_151019_unifyLDK/IH6x_11_05_112_1SS.csv
data/3-1_HukaData_151019_unifyLDK/IH6x_11_09_212_1SS.csv
data/3-1_HukaData_151019_unifyLDK/IH6x_11_09_112_1SS.csv
data/3-1_HukaData_151019_unifyLDK/IC6x_11_05_212_1SS.csv
data/3-1_HukaData_151019_unifyLDK/IC6x_11_05_112_1SS.csv
data/3-1_HukaData_151019_unifyLDK/IC6x_11_09_212_1SS.csv
data/3-1_HukaData_151019_unifyLDK/IC6x_11_09_112_1SS.csv
【暖房消費電力の計算】
	床面積の合計 (m2)							A_A = 120.08
	主たる居室の床面積 (m2)							A_MR = 29.81
	その他の居室の床面積 (m2)						A_OR = 51.34
	外皮の部位の面積の合計 (m2)						A_env = 307.51
	当該住戸の暖房期の日射取得係数 ((W/m2)/(W/m2))				mu_H = 0.11011767155229846
	当該住戸の冷房期の日射取得係数 ((W/m2)/(W/m2))				mu_C = 0.07170453031312457
	当該住戸の熱損失係数 (W/m2K)						Q = 2.647962191872085
	暖房方式								H_A = {'type': 'ダクト式セントラル空調機', 'duct_insulation': '全てもしくは一部が断熱区画外である', 'VAV': False, 'general_ventilation': True, 'EquipmentSpec': '入力しない'}
	暖房負荷								L_H_d_t_i
	冷房顕熱負荷								L_CS_d_t_i
	冷房潜熱負荷								L_

$ python samples\sample_central.py\
data\3-1_HukaData_151019_unifyLDK\IH6x_11_05_212_1SS.csv\
data\3-1_HukaData_151019_unifyLDK\IH6x_11_05_112_1SS.csv\
data\3-1_HukaData_151019_unifyLDK\IH6x_11_09_212_1SS.csv\
data\3-1_HukaData_151019_unifyLDK\IH6x_11_09_112_1SS.csv\
data\3-1_HukaData_151019_unifyLDK\IC6x_11_05_212_1SS.csv\
data\3-1_HukaData_151019_unifyLDK\IC6x_11_05_112_1SS.csv\
data\3-1_HukaData_151019_unifyLDK\IC6x_11_09_212_1SS.csv\
data\3-1_HukaData_151019_unifyLDK\IC6x_11_09_112_1SS.csv\
{'E_H': 36310.32799729332, 'E_C': 14746.052998129611}